In [133]:
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
import pymorphy2
# from collections import Counter, defaultdict

import re
import os

In [134]:
# Функция для получения списка адресов нужных файлов
def getting_documents():
    subtitles_directory = os.getcwd() + "\\friends"
    seasons_list = []
    documents_list = []
    # получение списка папок по сезонам
    for folder in os.listdir(subtitles_directory):
        seasons_list.append(subtitles_directory + '\\' + folder)
    # обход каждой папки с сезонами
    for one_folder in seasons_list:
        for text_file in os.listdir(one_folder):
            documents_list.append(one_folder + '\\' + text_file)
    # на выходе получается общий список
    return documents_list

In [135]:
# Настраиваем лемматизатор, устанавливаем стоп-слова и немного расширяем пунктуацию.
morph_analyze = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words("russian")
punctuation += '«»…—'

# Функция для препроцессинга текста
def preprocessing_text(text):
    lemmas = []
    # Переводим всё в нижний регистр шрифта и делим по пробелам
    pre_tokens = word_tokenize(text.lower())
    # Избавляемся от стоп-слов и пунктуации, лемматизируем
    for one_pre_token in pre_tokens:
        if one_pre_token not in punctuation and one_pre_token not in russian_stopwords:
            one_lemma = morph_analyze.normal_forms(one_pre_token)[0]
            lemmas.append(one_lemma)
    # Превращаем в строки с леммами, разделёнными пробелами
    processed_text = ' '.join(lemmas)
    return processed_text

In [137]:
# Вспомогательные списки для дальнейшего создания матриц
all_texts = []
all_episodes = []

for one_document in getting_documents():
    # Открываем по очереди все текстовые файлы и считываем текст из них
    with open(one_document, "r", encoding="utf-8") as f:
        original_text = f.read()
    # Проводим предпроцессинг текста
    processed_text = preprocessing_text(original_text)
    all_texts.append(processed_text)
    # Вытаскиваем из названия файла номер эпизода с помощью регулярного выражения
    episode_number = re.search(r'\dx\d+(-\d\d)*', one_document).group(0)
    all_episodes.append(episode_number)